# Подключаем пакеты и определяем функции

In [1]:
# Для работы с данными
import pandas as pd
import numpy as np
import wfdb
import ast
from utils import utils
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt   # plotting
#import seaborn as sns   # plotting heatmap

# Для работы с моделями
import tensorflow as tf
from tensorflow import keras
from keras import layers

# Для метрик
from keras import backend as K
from keras.metrics import AUC, Recall, Precision, Accuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score, roc_auc_score

tf.random.set_seed(42)
%matplotlib inline

# Скачиваем ICBEB с использованием кода и обработки авторов исследуемой статьи (обработанные данные, т.е. проведена нормализация и категоризация)

In [12]:
sampling_frequency=100
datafolder='data/ICBEB/'
#task = 'all'
#task='diagnostic'
#task='superdiagnostic'
#task = 'subdiagnostic'
#task = 'rhythm'
task = 'form'
outputfolder='output/'

# Load ICBEB data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# 1-9 for training 
X_train = data[labels.strat_fold < 10]
y_train = Y[labels.strat_fold < 10]
# 10 for validation
X_val = data[labels.strat_fold == 10]
y_val = Y[labels.strat_fold == 10]

# Стандартизация 3D данных c применением StandardScaler.
# Сначала изменяется форма данных а затем применяется нормализация. После этого требуется вернуть их прежнюю форму 
def standard_scaler(X_train, X_val): 
  scaler = StandardScaler()
  # Train
  num_instances, num_time_steps, num_features = X_train.shape
  X_train = np.reshape(X_train, newshape=(-1, num_features))
  X_train = scaler.fit_transform(X_train)
  X_train = np.reshape(X_train, newshape=(num_instances, num_time_steps, num_features))
    
  # Valid
  num_instances, num_time_steps, num_features = X_val.shape
  X_val = np.reshape(X_val, newshape=(-1, num_features))
  X_val = scaler.transform(X_val)
  X_val = np.reshape(X_val, newshape=(num_instances, num_time_steps, num_features))
  return X_train, X_val

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1523, 1000, 12), (1523, 3), (169, 1000, 12), (169, 3))

***task = 'all'***

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_all', X_train)
np.save('X_val_ICBEB_all', X_val)
np.save('y_train_ICBEB_all', y_train)
np.save('y_val_ICBEB_all', y_val)

##### task = 'diagnostic'

In [2]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [5]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_diag', X_train)
np.save('X_val_ICBEB_diag', X_val)
np.save('y_train_ICBEB_diag', y_train)
np.save('y_val_ICBEB_diag', y_val)

##### task = 'superdiagnostic'

In [4]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [7]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_superdiag', X_train)
np.save('X_val_ICBEB_superdiag', X_val)
np.save('y_train_ICBEB_superdiag', y_train)
np.save('y_val_ICBEB_superdiag', y_val)

##### task = 'subdiagnostic'

In [3]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [9]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_subdiag', X_train)
np.save('X_val_ICBEB_subdiag', X_val)
np.save('y_train_ICBEB_subdiag', y_train)
np.save('y_val_ICBEB_subdiag', y_val)

##### task = 'rhythm'

In [5]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [11]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_rhythm', X_train)
np.save('X_val_ICBEB_rhythm', X_val)
np.save('y_train_ICBEB_rhythm', y_train)
np.save('y_val_ICBEB_rhythm', y_val)

##### task = 'form'

In [7]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [13]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ICBEB_form', X_train)
np.save('X_val_ICBEB_form', X_val)
np.save('y_train_ICBEB_form', y_train)
np.save('y_val_ICBEB_form', y_val)

# Скачиваем PTB-XL с использованием кода и обработки авторов исследуемой статьи (обработанные данные, т.е. проведена нормализация и категоризация)

In [4]:
sampling_frequency=100
datafolder='data/ptbxl/'
task = 'all'
#task='diagnostic'
#task='superdiagnostic'
#task = 'subdiagnostic'
#task = 'rhythm'
#task = 'form'
outputfolder='output/'

# Load PTB-XL data
data, raw_labels = utils.load_dataset(datafolder, sampling_frequency)
# Preprocess label data
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
# Select relevant data and convert to one-hot
data, labels, Y, _ = utils.select_data(data, labels, task, min_samples=0, outputfolder=outputfolder)

# # 1-9 for training 
# X_train = data[labels.strat_fold < 10]
# y_train = Y[labels.strat_fold < 10]
# # 10 for validation
# X_val = data[labels.strat_fold == 10]
# y_val = Y[labels.strat_fold == 10]

# # Стандартизация 3D данных c применением StandardScaler.
# # Сначала изменяется форма данных а затем применяется нормализация. После этого требуется вернуть их прежнюю форму 
# def standard_scaler(X_train, X_val): 
#   scaler = StandardScaler()
#   # Train
#   num_instances, num_time_steps, num_features = X_train.shape
#   X_train = np.reshape(X_train, newshape=(-1, num_features))
#   X_train = scaler.fit_transform(X_train)
#   X_train = np.reshape(X_train, newshape=(num_instances, num_time_steps, num_features))
#   # Valid
#   num_instances, num_time_steps, num_features = X_val.shape
#   X_val = np.reshape(X_val, newshape=(-1, num_features))
#   X_val = scaler.transform(X_val)
#   X_val = np.reshape(X_val, newshape=(num_instances, num_time_steps, num_features))
#   return X_train, X_val

# X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [15]:
labels_test = labels[labels.strat_fold == 10]
labels_test

,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr,scp_codes_len,all_scp,all_scp_len
ecg_id,,,,,,,,,,,,,,,,,,,,,
9,18792.0,55.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-12-08 09:44:43,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,10,records100/00000/00009_lr,records500/00000/00009_hr,2,"[SR, NORM]",2
38,17076.0,40.0,0,NaN,72.0,2.0,0.0,CS-12 E,1985-02-15 11:48:22,sinusrhythmus schwierig bestimmbare qrs-achse,...,V5,NaN,NaN,NaN,10,records100/00000/00038_lr,records500/00000/00038_hr,2,"[SR, NORM]",2
40,19501.0,60.0,0,NaN,85.0,2.0,0.0,CS-12 E,1985-02-20 11:43:45,sinusrhythmus linkstyp sonst normales ekg,...,NaN,NaN,NaN,NaN,10,records100/00000/00040_lr,records500/00000/00040_hr,2,"[SR, NORM]",2
57,16063.0,26.0,0,NaN,93.0,2.0,0.0,CS-12 E,1985-06-06 11:32:43,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,10,records100/00000/00057_lr,records500/00000/00057_hr,2,"[SR, NORM]",2
59,19475.0,54.0,0,NaN,67.0,2.0,0.0,CS-12 E,1985-06-12 06:36:01,sinusrhythmus normales ekg,...,NaN,NaN,NaN,NaN,10,records100/00000/00059_lr,records500/00000/00059_hr,2,"[SR, NORM]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21809,12931.0,69.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-02-18 12:36:54,sinusrhythmus linkstyp qrs(t) abnorm inferi...,...,NaN,NaN,NaN,NaN,10,records100/21000/21809_lr,records500/21000/21809_hr,3,"[ABQRS, NORM, SR]",3
21812,20789.0,67.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-02-21 13:34:15,supraventrikulÄre arrhythmie a-v block i p-ver...,...,NaN,NaN,NaN,NaN,10,records100/21000/21812_lr,records500/21000/21812_hr,5,"[1AVB, CRBBB, LAFB, SVARR, ABQRS]",5
21818,19204.0,84.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-03-03 12:09:05,sinusrhythmus linkstyp mÄssige amplitudenkrite...,...,NaN,NaN,NaN,NaN,10,records100/21000/21818_lr,records500/21000/21818_hr,4,"[ABQRS, NORM, SR, VCLVH]",4


In [12]:
_

MultiLabelBinarizer()

In [16]:
labels_test.all_scp

ecg_id
9                               [SR, NORM]
38                              [SR, NORM]
40                              [SR, NORM]
57                              [SR, NORM]
59                              [SR, NORM]
                       ...                
21809                    [ABQRS, NORM, SR]
21812    [1AVB, CRBBB, LAFB, SVARR, ABQRS]
21818             [ABQRS, NORM, SR, VCLVH]
21819                    [IRBBB, LAFB, SR]
21826                  [IMI, ABQRS, SARRH]
Name: all_scp, Length: 2203, dtype: object

In [18]:
type(labels_test.all_scp)

pandas.core.series.Series

In [19]:
labels_test.all_scp.shape

(2203,)

In [21]:
import pandas as pd

# Разделение списков на отдельные строки
labels_expanded = labels_test.all_scp.explode()

# Подсчет количества вхождений каждого элемента
element_counts = labels_expanded.value_counts()

# Вывод результата
print(type(element_counts))

<class 'pandas.core.series.Series'>


In [23]:
# Вывод полной серии
pd.set_option('display.max_rows', None)
print(element_counts)

SR         1678
NORM        964
ABQRS       322
IMI         268
ASMI        236
LVH         214
NDT         183
LAFB        163
AFIB        152
ISC_        128
PVC         115
IRBBB       112
STD_        101
VCLVH        87
STACH        82
1AVB         80
IVCD         79
NST_         77
SARRH        77
ISCAL        66
SBRAD        64
QWAVE        55
CRBBB        54
CLBBB        54
ILMI         48
LOWT         44
NT_          42
LAO/LAE      42
PAC          40
AMI          35
LPR          34
INVT         29
PACE         29
ALMI         29
ISCIN        22
INJAS        22
LMI          20
LVOLT        18
ISCIL        18
DIG          18
LPFB         18
ISCAS        17
ISCLA        14
SVARR        14
INJAL        14
RVH          12
LNGQT        11
ANEUR        10
RAO/RAE      10
EL            9
WPW           8
ILBBB         8
BIGU          8
AFLT          7
HVOLT         6
IPLMI         5
ISCAN         4
SVTAC         3
STE_          3
SEHYP         3
TAB_          3
IPMI          3
3AVB    

In [25]:
from collections import Counter
import pandas as pd

# Соединение всех списков в один
combined_list = [item for sublist in labels_test.all_scp for item in sublist]

# Подсчет количества вхождений каждого элемента, сохраняя порядок следования
element_counts = pd.Series(dict(Counter(combined_list))).astype(int)

# Вывод результата
print(element_counts)

SR         1678
NORM        964
ABQRS       322
ASMI        236
IRBBB       112
SARRH        77
NDT         183
LAFB        163
NST_         77
LVH         214
ISC_        128
ISCAL        66
CRBBB        54
IMI         268
CLBBB        54
1AVB         80
SBRAD        64
STD_        101
PAC          40
VCLVH        87
INVT         29
LAO/LAE      42
AFIB        152
DIG          18
AFLT          7
PVC         115
SVTAC         3
LPR          34
LPFB         18
PACE         29
NT_          42
ILMI         48
IVCD         79
ANEUR        10
LOWT         44
ILBBB         8
ISCIL        18
QWAVE        55
ISCIN        22
INJAS        22
STACH        82
RAO/RAE      10
3AVB          2
INJIL         2
AMI          35
HVOLT         6
EL            9
TAB_          3
LMI          20
INJAL        14
LVOLT        18
BIGU          8
LNGQT        11
RVH          12
ALMI         29
ISCAS        17
STE_          3
ISCLA        14
WPW           8
SVARR        14
SEHYP         3
ISCAN         4
IPLMI   

***task = 'all'***

In [4]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_all', X_train)
np.save('X_val_ptbxl_all', X_val)
np.save('y_train_ptbxl_all', y_train)
np.save('y_val_ptbxl_all', y_val)

##### task = 'diagnostic'

In [8]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_diag', X_train)
np.save('X_val_ptbxl_diag', X_val)
np.save('y_train_ptbxl_diag', y_train)
np.save('y_val_ptbxl_diag', y_val)

##### task = 'superdiagnostic'

In [9]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_superdiag', X_train)
np.save('X_val_ptbxl_superdiag', X_val)
np.save('y_train_ptbxl_superdiag', y_train)
np.save('y_val_ptbxl_superdiag', y_val)

##### task = 'subdiagnostic'

In [10]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_subdiag', X_train)
np.save('X_val_ptbxl_subdiag', X_val)
np.save('y_train_ptbxl_subdiag', y_train)
np.save('y_val_ptbxl_subdiag', y_val)

##### task = 'rhythm'

In [11]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [4]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_rhythm', X_train)
np.save('X_val_ptbxl_rhythm', X_val)
np.save('y_train_ptbxl_rhythm', y_train)
np.save('y_val_ptbxl_rhythm', y_val)

##### task = 'form'

In [12]:
# # Выделим уникальные значения y_train в столбце и посчитаем их количество
# unique, counts = np.unique(y_train, return_counts = True, axis = 0) 
# print(unique, '\n', 'len(unique): ', len(unique))
# print(counts, '\n', 'len(counts): ', len(counts))

In [3]:
# Стандартизация данных
X_train, X_val = standard_scaler(X_train, X_val)

# Сохранение наборов данных в файлы .npy для дальнейшего использования в Google Colab
np.save('X_train_ptbxl_form', X_train)
np.save('X_val_ptbxl_form', X_val)
np.save('y_train_ptbxl_form', y_train)
np.save('y_val_ptbxl_form', y_val)

# Скачиваем ICBEB через форму PTB-XL

In [8]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename]
    data = [signal for signal, meta in data]
    for index, value in enumerate(data):
      if len(value) < 1000:
        data.remove(value)
        df = df.drop(index = index)
    refLen = 1000 # reference - эталон
    for index, value in enumerate(data):
      if len(value) > 1000:
        data[index] = data[index][:refLen]
    data = np.array(data)
    return data, df

path = 'data/ICBEB/'
sampling_rate=100

# load and convert annotation data
Y = pd.read_csv(path+'icbeb_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X, Y = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.form == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# # Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  data.remove(value)
C:\Users\manuk\AppData\Local\Temp\ipykernel_7528\110442798.py:7: DeprecationWarning: elementwise comparison failed;

In [9]:
X_train.shape, y_train.shape

((6181, 1000, 12), (6181,))

In [10]:
X_train[0]

array([[ 0.02822609,  0.00672717, -0.02150135, ..., -0.11200486,
        -0.59595824, -0.01558333],
       [ 0.05623843,  0.02373605, -0.03248586, ..., -0.09599664,
        -0.55091871,  0.03543248],
       [ 0.06525131,  0.05676969, -0.00847507, ..., -0.05695029,
        -0.48688198,  0.10148025],
       ...,
       [ 0.19612203, -0.16015955, -0.35627225, ...,  0.38222251,
         0.2396717 , -0.25792291],
       [-0.01297901, -0.48225593, -0.46927696, ..., -0.44072623,
        -0.56097927, -0.85331045],
       [-0.34466227, -0.59409148, -0.24942707, ..., -0.98916564,
        -1.00918153, -1.11767564]])

In [12]:
y_train

ecg_id
1          []
2          []
3          []
5          []
6          []
        ...  
6873       []
6874    [nan]
6875       []
6876       []
6877       []
Name: diagnostic_superclass, Length: 6181, dtype: object

In [13]:
Y

,filename,validation,age,sex,scp_codes,patient_id,quality,strat_fold,diagnostic_superclass
ecg_id,,,,,,,,,
1,records100/1,False,74.0,1,{'CRBBB': 100},1,0,7,[]
2,records100/2,False,49.0,0,{'NORM': 100},2,0,7,[]
3,records100/3,False,81.0,0,{'AFIB': 100},3,0,3,[]
4,records100/4,False,45.0,1,{'AFIB': 100},4,0,10,[]
5,records100/5,False,53.0,1,{'VPC': 100},5,0,7,[]
...,...,...,...,...,...,...,...,...,...
6873,records100/6873,False,80.0,1,{'1AVB': 100},6873,0,5,[]
6874,records100/6874,False,62.0,0,{'STD_': 100},6874,0,1,[nan]
6875,records100/6875,False,78.0,1,{'CLBBB': 100},6875,0,5,[]


In [15]:
# Создать Ndarray Numpy копию Series Pandas.
y_trainNp = y_train.to_numpy()
print(type(y_trainNp))
print(y_trainNp.size)

<class 'numpy.ndarray'>
6181


In [16]:
# Выделим уникальные значения y_train_np и посчитаем их количество.
unique, counts = np.unique(y_trainNp, return_counts = True)
print(unique, '\n', "unique.size: ", unique.size)
print(counts)

[list([]) list([nan])] 
 unique.size:  2
[4658 1523]


In [5]:
import pandas as pd
table = pd.read_csv('table_res_finetuning.csv', index_col=0)
table

,AUC,F2,G2
lstm_ICBEB_form,0.792765,0.498645,0.281553
lstm_bidir_ICBEB_form,0.895261,0.747274,0.512097
lstm_ptbxl_form,0.804124,0.041219,0.000000
lstm_bidir_ptbxl_form,0.802981,0.041219,0.000000
lstm_finetuning_form,0.691744,0.279402,0.000000
